In [1]:
using DataFrames, RDatasets, FixedEffectModels
using BenchmarkTools
using CSV, HTTP
using CategoricalArrays

In [2]:
function normalize_by_home_trade(df)
    
    home_trade = df[df.importer_index .== df.exporter_index,:].tradesharedata
    # grabs the home trade share by finding where importer = exporter
    
    df.tradesharedata .= df.tradesharedata / home_trade
    # normalize things
    
    return df
    
end 

# grpek = groupby(dfektrade , "importer_index");
# # a grouping operation

# # foo[1][foo[1].importer_index .== foo[1].exporter_index,:]

# dfnormtrade = combine(normalize_by_home_trade, grpek);

In [4]:
dfekimporter = CSV.read("../ek-data/ek-importer.csv", DataFrame, header = 0)

rename!(dfekimporter,[:importer])

dfekexporter = CSV.read("../ek-data/ek-exporter.csv", DataFrame, header = 0)

rename!(dfekexporter,[:exporter])

dfektrade = CSV.read("../ek-data/ek-alt-trade.csv", DataFrame, header = 0)

rename!(dfektrade,[:trade])

dfekdistance = CSV.read("../ek-data/ek-distance.csv", DataFrame, header = 0)

rename!(dfekdistance,[:distance])

dfaltekdistance = CSV.read("../ek-data/ek-alt-distance.csv", DataFrame, header = 0)

rename!(dfaltekdistance,[:bin375,:bin750,:bin1500,:bin3000,:bin6000,:binmax])

dfekdistance.distance = 10.0*dfekdistance.distance

dfekborder = CSV.read("../ek-data/ek-border.csv", DataFrame, header = 0)

rename!(dfekborder,[:border])

dfeklanguage = CSV.read("../ek-data/ek-language-common-area.csv", DataFrame, header = 0)

rename!(dfeklanguage,[:sharedlanguage,:europeancom,:efta])

println(dfeklanguage[1:10,:])

10×3 DataFrame
 Row │ sharedlanguage  europeancom  efta  
     │ Int64           Int64        Int64 
─────┼────────────────────────────────────
   1 │              0            0      0
   2 │              0            0      0
   3 │              1            0      0
   4 │              0            0      0
   5 │              0            0      0
   6 │              0            0      0
   7 │              0            0      0
   8 │              0            0      0
   9 │              0            0      0
  10 │              0            0      0


In [7]:
dfaltekdistance.distbin .= 0.0;

dfaltekdistance[:, :distbin] .= ifelse.(dfaltekdistance.bin375 .== 1, 1.0, dfaltekdistance[:, :distbin]);

dfaltekdistance[:, :distbin] .= ifelse.((dfaltekdistance.bin750 .== 1 ), 2.0, dfaltekdistance[:, :distbin]);

dfaltekdistance[:, :distbin] .= ifelse.((dfaltekdistance.bin1500 .== 1 ), 3.0, dfaltekdistance[:, :distbin]);

dfaltekdistance[:, :distbin] .= ifelse.((dfaltekdistance.bin3000 .== 1 ), 4.0, dfaltekdistance[:, :distbin]);

dfaltekdistance[:, :distbin] .= ifelse.((dfaltekdistance.bin6000 .== 1 ), 5.0, dfaltekdistance[:, :distbin]);

dfaltekdistance[:, :distbin] .= ifelse.((dfaltekdistance.binmax .== 1 ), 6.0, dfaltekdistance[:, :distbin]);

dfektrade = hcat(dfektrade , dfekdistance)

dfektrade = hcat(dfektrade , dfekborder)

dfektrade = hcat(dfektrade, dfaltekdistance)

dfektrade = hcat(dfektrade, dfekimporter)

dfektrade = hcat(dfektrade, dfekexporter)

println(dfektrade[1:20, :])



20×12 DataFrame
 Row │ trade    distance  border  bin375  bin750  bin1500  bin3000  bin6000  binmax  distbin  importer  exporter 
     │ Float64  Float64   Int64   Int64   Int64   Int64    Int64    Int64    Int64   Float64  Int64     Int64    
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │   0.0         0.0       0       1       0        0        0        0       0      1.0         1         1
   2 │  -7.41     9891.0       0       0       0        0        0        0       1      6.0         1         2
   3 │ -10.074   10389.0       0       0       0        0        0        0       1      6.0         1         3
   4 │  -5.826   10009.0       0       0       0        0        0        0       1      6.0         1         4
   5 │  -8.206    9968.0       0       0       0        0        0        0       1      6.0         1         5
   6 │  -6.461    9453.0       0       0       0        0        0        0  

In [10]:
sum(dfektrade.distbin .== 6.0)

76

In [11]:
CSV.write("ek-data.csv", dfektrade)

CSV.write("ek-language.csv", dfeklanguage)

"ek-language.csv"

In [36]:

filter!(row -> ~(row.trade ≈ 1.0), dfektrade);

filter!(row -> ~(row.trade ≈ 0.0), dfektrade);

size(dfektrade)


(342, 12)

In [37]:
dfektrade = hcat(dfektrade, dfeklanguage);

In [38]:
println(dfektrade[1:19,:])

19×15 DataFrame
 Row │ trade    distance  distbin  border  bin375  bin750  bin1500  bin3000  bin6000  binmax  importer  exporter  sharedlanguage  europeancom  efta  
     │ Float64  Float64   Float64  Int64   Int64   Int64   Int64    Int64    Int64    Int64   Int64     Int64     Int64           Int64        Int64 
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │  -7.41     9891.0      6.0       0       0       0        0        0        0       1         1         2               0            0      0
   2 │ -10.074   10389.0      6.0       0       0       0        0        0        0       1         1         3               0            0      0
   3 │  -5.826   10009.0      6.0       0       0       0        0        0        0       1         1         4               1            0      0
   4 │  -8.206    9968.0      6.0       0       0       0        0        0        0   

In [39]:
# don't have distance in here, but this the basic regression

@time outreg = reg(dfektrade, @formula(trade ~ fe(importer) + fe(exporter) +
         bin375 + bin750 + bin1500 + bin3000 + bin6000 + binmax  + border + 
                sharedlanguage + europeancom + efta + 0), save = true)

# in this one, intercept of not does not matter
# but if one does fe on the distance bin, the border stuff etc. won't match up

  0.884772 seconds (2.25 M allocations: 122.481 MiB, 2.73% gc time, 99.39% compilation time)


                            Fixed Effect Model                            
Number of obs:                   342  Degrees of freedom:                8
R2:                            0.976  R2 Adjusted:                   0.972
F-Stat:                      87.9716  p-value:                       0.000
R2 within:                     0.729  Iterations:                        4
trade          |  Estimate Std.Error  t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------------------
bin375         |   3.45649  0.204755  16.8811    0.000   3.05352   3.85946
bin750         |   2.89299  0.167308  17.2914    0.000   2.56372   3.22227
bin1500        |   2.52554  0.156294  16.1589    0.000   2.21794   2.83313
bin3000        |   2.34086   0.20072  11.6623    0.000   1.94583   2.73589
bin6000        |  0.494561  0.121258  4.07857    0.000  0.255917  0.733205
binmax         |       0.0       NaN      NaN      NaN       NaN       NaN
border         |  0.30356

The basic issue is this. First, the normal stuff matches up, border, language, etc. with EK02. Next, the levels of the distance bins do not match up...but the relative values do. So the effect from moving closest to furthest away is -3.46 which is exactly what is implied by EK02.

Then notice that the exporter fixed effect lines up with $S_i$s in EK. 

So what is happening (for some reason) the importer fe are not normalized

In [53]:
reg(dfektrade, @formula(trade ~ fe(importer) + fe(exporter) +
         bin375 + bin750 + bin1500 + bin3000 + bin6000 + binmax  + border + 
                sharedlanguage + europeancom + efta + 0))

                            Fixed Effect Model                            
Number of obs:                   342  Degrees of freedom:                8
R2:                            0.976  R2 Adjusted:                   0.972
F-Stat:                      87.9716  p-value:                       0.000
R2 within:                     0.729  Iterations:                        4
trade          |  Estimate Std.Error  t value Pr(>|t|) Lower 95% Upper 95%
--------------------------------------------------------------------------
bin375         |   3.45649  0.204755  16.8811    0.000   3.05352   3.85946
bin750         |   2.89299  0.167308  17.2914    0.000   2.56372   3.22227
bin1500        |   2.52554  0.156294  16.1589    0.000   2.21794   2.83313
bin3000        |   2.34086   0.20072  11.6623    0.000   1.94583   2.73589
bin6000        |  0.494561  0.121258  4.07857    0.000  0.255917  0.733205
binmax         |       0.0       NaN      NaN      NaN       NaN       NaN
border         |  0.30356

In [42]:
outreg.coef[1:6] .+ sum(unique(outreg.fe.fe_importer)) / 19

6-element Vector{Float64}:
 -3.102440151020191
 -3.665940509065175
 -4.03339539351596
 -4.218077290075897
 -6.064372105694248
 -6.558933104672115

In [45]:
println(outreg.fe[1:20,:])

20×4 DataFrame
 Row │ importer  exporter  fe_importer  fe_exporter 
     │ Int64     Int64     Float64?     Float64?    
─────┼──────────────────────────────────────────────
   1 │        1         2     -6.51527  -1.16184
   2 │        1         3     -6.51527  -3.33563
   3 │        1         4     -6.51527   0.411561
   4 │        1         5     -6.51527  -1.74955
   5 │        1         6     -6.51527  -0.522466
   6 │        1         7     -6.51527   1.28138
   7 │        1         8     -6.51527   2.35275
   8 │        1         9     -6.51527  -2.81352
   9 │        1        10     -6.51527   1.78151
  10 │        1        11     -6.51527   4.19879
  11 │        1        12     -6.51527  -2.18932
  12 │        1        13     -6.51527  -1.19769
  13 │        1        14     -6.51527  -1.34615
  14 │        1        15     -6.51527  -1.57318
  15 │        1        16     -6.51527   0.303089
  16 │        1        17     -6.51527   0.00981614
  17 │        1        18     -6.515

In [48]:
unique(outreg.fe.fe_exporter)

19-element Vector{Union{Missing, Float64}}:
 -1.1618390691970162
 -3.3356251102144108
  0.41156104550239414
 -1.7495500129450563
 -0.5224659009637302
  1.281384026967499
  2.352748237706335
 -2.813521274291235
  1.7815099846745037
  4.198787040343374
 -2.1893194485603877
 -1.1976868887919667
 -1.3461517689525517
 -1.5731796130646756
  0.30308906006441294
  0.009816140326315725
  1.3741669179055849
  3.9837189402392372
  0.19255769325137706

In [40]:
unique(outreg.fe.fe_importer) .- sum(unique(outreg.fe.fe_importer)) / 19

# notice this, if we normalize the numay variables by this
# this is exactly the estimate of the bin6

19-element Vector{Float64}:
  0.04366295640927209
 -0.5165759113022785
  4.455743310838223
  0.27703472971292076
  1.243660513370732
 -0.812021877891218
 -1.0630896572430286
 -1.3539359728199818
  0.45711030465613245
 -1.7113321205886551
 -2.613897170182944
  3.1941016040711925
  1.264260479629086
  0.34574296788955383
  0.36639933430374594
 -1.4597004136197986
 -0.034680514325143186
 -0.5573593978838884
 -1.5251231650239232

In [346]:
-1.16 + 1.68


0.52